In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier

# path1 = '/home/jian/DATA_SETS/kaggle/forests/train.csv'
# path2 = '/home/jian/DATA_SETS/kaggle/forests/test.csv'

path1 = 'all/train-set.csv'
path2 = 'all/test-set.csv'

submission = 'submission.csv'


def preprocess(data2):
    data = data2
    feature_cols_for_filling_missing = [col for col in data.columns if col not in ['Hillshade_3pm', 'Id']]
    X_train = data[feature_cols_for_filling_missing][data.Hillshade_3pm != 0]
    y_train = data['Hillshade_3pm'][data.Hillshade_3pm != 0]
    X_test = data[feature_cols_for_filling_missing][data.Hillshade_3pm == 0]
    from sklearn.ensemble import RandomForestRegressor

    rfg = RandomForestRegressor()
    rfg.fit(X_train, y_train)
    data.Hillshade_3pm.loc[data.Hillshade_3pm == 0] = np.around(rfg.predict(X_test))
    return data


def feature_engineering(data2):
    data = data2

    data['Ele_minus_VDtHyd'] = data.Elevation - data.Vertical_Distance_To_Hydrology

    data['Ele_plus_VDtHyd'] = data.Elevation + data.Vertical_Distance_To_Hydrology

    data['Distanse_to_Hydrolody'] = (data['Horizontal_Distance_To_Hydrology'] ** 2 + data[
        'Vertical_Distance_To_Hydrology'] ** 2) ** 0.5

    data['Hydro_plus_Fire'] = data['Horizontal_Distance_To_Hydrology'] + data['Horizontal_Distance_To_Fire_Points']

    data['Hydro_minus_Fire'] = data['Horizontal_Distance_To_Hydrology'] - data['Horizontal_Distance_To_Fire_Points']

    data['Hydro_plus_Road'] = data['Horizontal_Distance_To_Hydrology'] + data['Horizontal_Distance_To_Roadways']

    data['Hydro_minus_Road'] = data['Horizontal_Distance_To_Hydrology'] - data['Horizontal_Distance_To_Roadways']

    data['Fire_plus_Road'] = data['Horizontal_Distance_To_Fire_Points'] + data['Horizontal_Distance_To_Roadways']

    data['Fire_minus_Road'] = data['Horizontal_Distance_To_Fire_Points'] - data['Horizontal_Distance_To_Roadways']

    data['Soil'] = 0
    for i in range(1, 41):
        data['Soil'] = data['Soil'] + i * data['Soil_Type' + str(i)]

    data['Wilderness_Area'] = 0
    for i in range(1, 5):
        data['Wilderness_Area'] = data['Wilderness_Area'] + i * data['Wilderness_Area' + str(i)]

    return data


def get_features():
    return ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
            'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
            'Horizontal_Distance_To_Fire_Points',
            'Ele_minus_VDtHyd', 'Ele_plus_VDtHyd', 'Distanse_to_Hydrolody', 'Hydro_plus_Fire', 'Hydro_minus_Fire',
            'Hydro_plus_Road',
            'Hydro_minus_Road', 'Fire_plus_Road', 'Fire_minus_Road', 'Soil', 'Wilderness_Area']


def main():
    train_df = pd.read_csv(path1)
    test_df = pd.read_csv(path2)

#     train_df = preprocess(train_df)
#     test_df = preprocess(test_df)

    train_df = feature_engineering(train_df)
    test_df = feature_engineering(test_df)

    features = get_features()

    y_train = train_df['Cover_Type'].values
    test_id = test_df['Id']
    X_train = train_df[:][features].values

    # X_test = test_df[:][features]

    def split_X_test():

        length = len(test_df)
        n = length // 10000
        print(n)
        split_test_data = []

        for i in range(n):
            split_test_data.append(test_df[i * 10000:(i + 1) * 10000][features].values)

        split_test_data.append(test_df[n * 10000:length][features].values)

        return split_test_data

    X_test = split_X_test()

    print('Start')

    clf = ExtraTreesClassifier(max_features=0.3, n_estimators=500)
    clf.fit(X_train, y_train)

    print('train over')

    y_predict = []

    for var in X_test:
        y_predict.extend(clf.predict(var))

    submission = pd.DataFrame(data={'Id': test_id, 'Cover_Type': y_predict})
    submission.to_csv("submission.csv", index=False)

    print('Compelte')


if __name__ == '__main__':
    main()

22
Start
train over
Compelte


In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier

# path1 = '/home/jian/DATA_SETS/kaggle/forests/train.csv'
# path2 = '/home/jian/DATA_SETS/kaggle/forests/test.csv'

path1 = 'all/train-set.csv'
path2 = 'all/test-set.csv'

submission = 'submission.csv'

In [4]:
def preprocess(data2):
    data = data2
    feature_cols_for_filling_missing = [col for col in data.columns if col not in ['Hillshade_3pm', 'Id']]
    X_train = data[feature_cols_for_filling_missing][data.Hillshade_3pm != 0]
    y_train = data['Hillshade_3pm'][data.Hillshade_3pm != 0]
    X_test = data[feature_cols_for_filling_missing][data.Hillshade_3pm == 0]
    from sklearn.ensemble import RandomForestRegressor

    rfg = RandomForestRegressor()
    rfg.fit(X_train, y_train)
    data.Hillshade_3pm.loc[data.Hillshade_3pm == 0] = np.around(rfg.predict(X_test))
    return data
train_df = pd.read_csv(path1)
test_df = pd.read_csv(path2)

train_df = preprocess(train_df)
test_df = preprocess(test_df)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528720 entries, 0 to 528719
Data columns (total 56 columns):
Id                                    528720 non-null int64
Elevation                             528720 non-null int64
Aspect                                528720 non-null int64
Slope                                 528720 non-null int64
Horizontal_Distance_To_Hydrology      528720 non-null int64
Vertical_Distance_To_Hydrology        528720 non-null int64
Horizontal_Distance_To_Roadways       528720 non-null int64
Hillshade_9am                         528720 non-null int64
Hillshade_Noon                        528720 non-null int64
Hillshade_3pm                         528720 non-null float64
Horizontal_Distance_To_Fire_Points    528720 non-null int64
Wilderness_Area1                      528720 non-null int64
Wilderness_Area2                      528720 non-null int64
Wilderness_Area3                      528720 non-null int64
Wilderness_Area4                      528720 no

In [5]:
train_df = pd.read_csv(path1)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528720 entries, 0 to 528719
Data columns (total 56 columns):
Id                                    528720 non-null int64
Elevation                             528720 non-null int64
Aspect                                528720 non-null int64
Slope                                 528720 non-null int64
Horizontal_Distance_To_Hydrology      528720 non-null int64
Vertical_Distance_To_Hydrology        528720 non-null int64
Horizontal_Distance_To_Roadways       528720 non-null int64
Hillshade_9am                         528720 non-null int64
Hillshade_Noon                        528720 non-null int64
Hillshade_3pm                         528720 non-null int64
Horizontal_Distance_To_Fire_Points    528720 non-null int64
Wilderness_Area1                      528720 non-null int64
Wilderness_Area2                      528720 non-null int64
Wilderness_Area3                      528720 non-null int64
Wilderness_Area4                      528720 non-

In [11]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier

train_df = pd.read_csv(path1)
test_df = pd.read_csv(path2)

# train_df = preprocess(train_df)
# test_df = preprocess(test_df)

def feature_change(data_input):
    data = data_input
    data['Ele_minus_VDtHyd'] = data.Elevation - data.Vertical_Distance_To_Hydrology
    data['Ele_plus_VDtHyd'] = data.Elevation + data.Vertical_Distance_To_Hydrology
    data['Distanse_to_Hydrolody'] = (data['Horizontal_Distance_To_Hydrology'] ** 2 + data['Vertical_Distance_To_Hydrology'] ** 2) ** 0.5
    data['Hydro_plus_Fire'] = data['Horizontal_Distance_To_Hydrology'] + data['Horizontal_Distance_To_Fire_Points']
    data['Hydro_minus_Fire'] = data['Horizontal_Distance_To_Hydrology'] - data['Horizontal_Distance_To_Fire_Points']
    data['Hydro_plus_Road'] = data['Horizontal_Distance_To_Hydrology'] + data['Horizontal_Distance_To_Roadways']
    data['Hydro_minus_Road'] = data['Horizontal_Distance_To_Hydrology'] - data['Horizontal_Distance_To_Roadways']
    data['Fire_plus_Road'] = data['Horizontal_Distance_To_Fire_Points'] + data['Horizontal_Distance_To_Roadways']
    data['Fire_minus_Road'] = data['Horizontal_Distance_To_Fire_Points'] - data['Horizontal_Distance_To_Roadways']
    data['Soil'] = 0
    for i in range(1, 41):
        data['Soil'] = data['Soil'] + i * data['Soil_Type' + str(i)]
    data['Wilderness_Area'] = 0
    for i in range(1, 5):
        data['Wilderness_Area'] = data['Wilderness_Area'] + i * data['Wilderness_Area' + str(i)]
    for i in range(1, 41):
        data = data.drop(['Soil_Type' + str(i)], axis=1)
    for i in range(1, 5):
        data = data.drop(['Wilderness_Area' + str(i)], axis=1)
    return data

def get_features():
    return ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
            'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
            'Horizontal_Distance_To_Fire_Points',
            'Ele_minus_VDtHyd', 'Ele_plus_VDtHyd', 'Distanse_to_Hydrolody', 'Hydro_plus_Fire', 'Hydro_minus_Fire',
            'Hydro_plus_Road',
            'Hydro_minus_Road', 'Fire_plus_Road', 'Fire_minus_Road', 'Soil', 'Wilderness_Area']

train_df = feature_change(train_df)
test_df = feature_change(test_df)

features = get_features()

y_train = train_df['Cover_Type'].values
test_id = test_df['Id']
X_train = train_df[:][features].values

def split_X_test():

    length = len(test_df)
    n = length // 10000
    print(n)
    split_test_data = []

    for i in range(n):
        split_test_data.append(test_df[i * 10000:(i + 1) * 10000][features].values)

    split_test_data.append(test_df[n * 10000:length][features].values)

    return split_test_data

X_test = split_X_test()

22


In [18]:
X_test = test_df[:][features].values
print(len(X_train[1]))

21


In [19]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier

train_df = pd.read_csv(path1)
test_df = pd.read_csv(path2)

# train_df = preprocess(train_df)
# test_df = preprocess(test_df)

def feature_change(data_input):
    data = data_input
    data['Ele_minus_VDtHyd'] = data.Elevation - data.Vertical_Distance_To_Hydrology
    data['Ele_plus_VDtHyd'] = data.Elevation + data.Vertical_Distance_To_Hydrology
    data['Distanse_to_Hydrolody'] = (data['Horizontal_Distance_To_Hydrology'] ** 2 + data['Vertical_Distance_To_Hydrology'] ** 2) ** 0.5
    data['Hydro_plus_Fire'] = data['Horizontal_Distance_To_Hydrology'] + data['Horizontal_Distance_To_Fire_Points']
    data['Hydro_minus_Fire'] = data['Horizontal_Distance_To_Hydrology'] - data['Horizontal_Distance_To_Fire_Points']
    data['Hydro_plus_Road'] = data['Horizontal_Distance_To_Hydrology'] + data['Horizontal_Distance_To_Roadways']
    data['Hydro_minus_Road'] = data['Horizontal_Distance_To_Hydrology'] - data['Horizontal_Distance_To_Roadways']
    data['Fire_plus_Road'] = data['Horizontal_Distance_To_Fire_Points'] + data['Horizontal_Distance_To_Roadways']
    data['Fire_minus_Road'] = data['Horizontal_Distance_To_Fire_Points'] - data['Horizontal_Distance_To_Roadways']
    data['Soil'] = 0
    for i in range(1, 41):
        data['Soil'] = data['Soil'] + i * data['Soil_Type' + str(i)]
    data['Wilderness_Area'] = 0
    for i in range(1, 5):
        data['Wilderness_Area'] = data['Wilderness_Area'] + i * data['Wilderness_Area' + str(i)]
    for i in range(1, 41):
        data = data.drop(['Soil_Type' + str(i)], axis=1)
    for i in range(1, 5):
        data = data.drop(['Wilderness_Area' + str(i)], axis=1)
    return data

def get_features():
    return ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
            'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
            'Horizontal_Distance_To_Fire_Points',
            'Ele_minus_VDtHyd', 'Ele_plus_VDtHyd', 'Distanse_to_Hydrolody', 'Hydro_plus_Fire', 'Hydro_minus_Fire',
            'Hydro_plus_Road',
            'Hydro_minus_Road', 'Fire_plus_Road', 'Fire_minus_Road', 'Soil', 'Wilderness_Area']

train_df = feature_change(train_df)
test_df = feature_change(test_df)

features = get_features()

y_train = train_df['Cover_Type'].values
test_id = test_df['Id']
X_train = train_df[:][features].values
X_test = test_df[:][features].values

# def split_X_test():

#     length = len(test_df)
#     n = length // 10000
#     print(n)
#     split_test_data = []

#     for i in range(n):
#         split_test_data.append(test_df[i * 10000:(i + 1) * 10000][features].values)

#     split_test_data.append(test_df[n * 10000:length][features].values)

#     return split_test_data

# X_test = split_X_test()

print('Start')

clf = ExtraTreesClassifier(max_features=0.3, n_estimators=500)
clf.fit(X_train, y_train)

print('train over')

y_predict = []

for var in X_test:
    y_predict.extend(clf.predict(var))

submission = pd.DataFrame(data={'Id': test_id, 'Cover_Type': y_predict})
submission.to_csv("submission.csv", index=False)

print('Compelte')

Start
train over


ValueError: Expected 2D array, got 1D array instead:
array=[ 2.9530000e+03  2.2400000e+02  9.0000000e+00  5.5000000e+01
  1.1000000e+01  1.6000000e+03  2.0600000e+02  2.5100000e+02
  1.7600000e+02  1.6690000e+03  2.9420000e+03  2.9640000e+03
  5.6089214e+01  1.7240000e+03 -1.6140000e+03  1.6550000e+03
 -1.5450000e+03  3.2690000e+03  6.9000000e+01  3.2000000e+01
  3.0000000e+00].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [20]:
y_predict = clf.predict(X_test)


submission = pd.DataFrame(data={'Id': test_id, 'Cover_Type': y_predict})
submission.to_csv("submission.csv", index=False)

print('Compelte')

Compelte


In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier

train_df = pd.read_csv('all/train-set.csv')
test_df = pd.read_csv('all/test-set.csv')

def change_feature(data_input):
    data = data_input
    data['Ele_minus_VDtHyd'] = data.Elevation - data.Vertical_Distance_To_Hydrology
    data['Ele_plus_VDtHyd'] = data.Elevation + data.Vertical_Distance_To_Hydrology
    data['Distanse_to_Hydrolody'] = (data['Horizontal_Distance_To_Hydrology'] ** 2 + data['Vertical_Distance_To_Hydrology'] ** 2) ** 0.5
    data['Hydro_plus_Fire'] = data['Horizontal_Distance_To_Hydrology'] + data['Horizontal_Distance_To_Fire_Points']
    data['Hydro_minus_Fire'] = data['Horizontal_Distance_To_Hydrology'] - data['Horizontal_Distance_To_Fire_Points']
    data['Hydro_plus_Road'] = data['Horizontal_Distance_To_Hydrology'] + data['Horizontal_Distance_To_Roadways']
    data['Hydro_minus_Road'] = data['Horizontal_Distance_To_Hydrology'] - data['Horizontal_Distance_To_Roadways']
    data['Fire_plus_Road'] = data['Horizontal_Distance_To_Fire_Points'] + data['Horizontal_Distance_To_Roadways']
    data['Fire_minus_Road'] = data['Horizontal_Distance_To_Fire_Points'] - data['Horizontal_Distance_To_Roadways']
    data['Soil'] = 0
    for i in range(1, 41):
        data['Soil'] = data['Soil'] + i * data['Soil_Type' + str(i)]
    data['Wilderness_Area'] = 0
    for i in range(1, 5):
        data['Wilderness_Area'] = data['Wilderness_Area'] + i * data['Wilderness_Area' + str(i)]
    for i in range(1, 41):
        data = data.drop(['Soil_Type' + str(i)], axis=1)
    for i in range(1, 5):
        data = data.drop(['Wilderness_Area' + str(i)], axis=1)
    return data

def get_features():
    return ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
            'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
            'Horizontal_Distance_To_Fire_Points',
            'Ele_minus_VDtHyd', 'Ele_plus_VDtHyd', 'Distanse_to_Hydrolody', 'Hydro_plus_Fire', 'Hydro_minus_Fire',
            'Hydro_plus_Road',
            'Hydro_minus_Road', 'Fire_plus_Road', 'Fire_minus_Road', 'Soil', 'Wilderness_Area']

train_df = change_feature(train_df)
test_df = change_feature(test_df)

features = get_features()

y_train = train_df['Cover_Type'].values
test_id = test_df['Id']
X_train = train_df[:][features].values
X_test = test_df[:][features].values


print('Start')

clf = ExtraTreesClassifier(max_features=0.3, n_estimators=500)
clf.fit(X_train, y_train)

print('train over')

output = clf.predict(X_test)


# submission = pd.DataFrame(data={'Id': test_id, 'Cover_Type': y_predict})
# submission.to_csv("submission.csv", index=False)



result = np.c_[test_id.astype(int), output.astype(int)]
df_result = pd.DataFrame(result[:,0:2], columns=['Id', 'Cover_Type'])
df_result.to_csv('all/forest.csv', index=False)

print('Compelte')





Start
train over
Compelte
